In [1]:
import pandas as pd
import os
import re
import cfg

In [9]:
COINS = ['1000BONKUSDT',
    '1000FLOKIUSDT',
    '1000PEPEUSDT',
    '1000SHIBUSDT',
    'AAVEUSDT',
    'ADAUSDT',
    'APEUSDT',
    'APTUSDT',
    'ARBUSDT',
    'ARUSDT',
    'ATOMUSDT',
    'AVAXUSDT',
    'AXSUSDT',
    'BCHUSDT',
    'BNBUSDT',
    'BTCUSDT',
    'CRVUSDT',
    'DOGEUSDT',
    'DOTUSDT',
    'DYDXUSDT',
    'ENAUSDT',
    'ETCUSDT',
    'ETHUSDT',
    'FETUSDT',
    'FILUSDT',
    'GALAUSDT',
    'INJUSDT',
    'JTOUSDT',
    'JUPUSDT',
    'LDOUSDT',
    'LINKUSDT',
    'LTCUSDT',
    'MKRUSDT',
    'NEARUSDT',
    'OPUSDT',
    'ORDIUSDT',
    'PENDLEUSDT',
    'POLUSDT',
    'PYTHUSDT',
    'RUNEUSDT',
    'SANDUSDT',
    'SEIUSDT',
    'SOLUSDT',
    'STXUSDT',
    'SUIUSDT',
    'TAOUSDT',
    'TIAUSDT',
    'TONUSDT',
    'TRXUSDT',
    'UNIUSDT',
    'WIFUSDT',
    'WLDUSDT',
    'XRPUSDT',
    'RENDERUSDT'
    ]
COINS_SPOT = [coin.replace("1000","") for coin in COINS]

In [3]:
## check the number of symbols for bn spot 1min bar for each date

In [4]:
path = '/data/crypto/bar1m/spot/'
files = os.listdir(path)
print(len(files))


340


In [5]:
rows = []

for symbol in files:
    df = pd.read_parquet(f"{path}/{symbol}")
    df["end_tm"] = pd.to_datetime(df["end_tm"])
    unique_dates = df["end_tm"].dt.normalize().unique()
    unique_dates = pd.to_datetime(unique_dates)

    start_dt = unique_dates.min()
    end_dt = unique_dates.max()

    full_range = pd.date_range(start=start_dt, end=end_dt, freq="D")

    missing = sorted(set(full_range) - set(unique_dates))

    rows.append({
        "coin": symbol[:-8],   # strip .parquet
        "start_dt": start_dt,
        "end_dt": end_dt,
        "unique_days": len(unique_dates),
        "expected_days": len(full_range),
        "missing_days": len(missing),
        "missing_dates": [d.strftime("%Y-%m-%d") for d in missing]  # readable
    })

df_summary = pd.DataFrame(rows)



In [6]:
df_summary[df_summary.missing_days!=0]

,coin,start_dt,end_dt,unique_days,expected_days,missing_days,missing_dates
19,QUICKUSDT,2021-08-13,2025-07-15,1430,1433,3,"[2023-07-18, 2023-07-19, 2023-07-20]"
102,CVCUSDT,2020-01-01,2025-07-14,1869,2022,153,"[2022-12-10, 2022-12-11, 2022-12-12, 2022-12-1..."
181,SUNUSDT,2020-09-21,2025-07-15,1756,1759,3,"[2021-06-15, 2021-06-16, 2021-06-17]"
316,USDCUSDT,2020-01-01,2025-07-15,1858,2023,165,"[2022-09-27, 2022-09-28, 2022-09-29, 2022-09-3..."


In [11]:
# 
import cfg
from bn_downloader import *
from datetime import datetime
start = pd.to_datetime('2020-01-01')
end = datetime.now()
data_dir = '/data/crypto/'
for idx, row in df_summary[df_summary.missing_days!=0].iterrows():
    #break
    dates = row['missing_dates']
    download_daily_klines('spot', [row['coin']], 1, ['1m'], dates, start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), cfg.bar1m_folder, True)

Found 1 symbols
[1/1] - start download daily QUICKUSDT klines 

File not found: https://data.binance.vision/data/spot/daily/klines/QUICKUSDT/1m/QUICKUSDT-1m-2023-07-18.zip

File not found: https://data.binance.vision/data/spot/daily/klines/QUICKUSDT/1m/QUICKUSDT-1m-2023-07-18.zip.CHECKSUM

File not found: https://data.binance.vision/data/spot/daily/klines/QUICKUSDT/1m/QUICKUSDT-1m-2023-07-19.zip

File not found: https://data.binance.vision/data/spot/daily/klines/QUICKUSDT/1m/QUICKUSDT-1m-2023-07-19.zip.CHECKSUM

File not found: https://data.binance.vision/data/spot/daily/klines/QUICKUSDT/1m/QUICKUSDT-1m-2023-07-20.zip

File not found: https://data.binance.vision/data/spot/daily/klines/QUICKUSDT/1m/QUICKUSDT-1m-2023-07-20.zip.CHECKSUM
Found 1 symbols
[1/1] - start download daily CVCUSDT klines 

File not found: https://data.binance.vision/data/spot/daily/klines/CVCUSDT/1m/CVCUSDT-1m-2022-12-10.zip

File not found: https://data.binance.vision/data/spot/daily/klines/CVCUSDT/1m/CVCUSDT-1m-

['2023-07-18', '2023-07-19', '2023-07-20']

In [7]:
df_summary[df_summary.coin.isin(COINS_SPOT)]

,coin,start_dt,end_dt,unique_days,expected_days,missing_days,missing_dates
4,TRXUSDT,2020-01-01,2025-08-17,2056,2056,0,[]
23,RENDERUSDT,2024-07-26,2025-08-17,388,388,0,[]
24,JUPUSDT,2024-01-31,2025-08-17,565,565,0,[]
50,INJUSDT,2020-10-21,2025-08-17,1762,1762,0,[]
55,AXSUSDT,2020-11-04,2025-08-17,1748,1748,0,[]
61,XRPUSDT,2020-01-01,2025-08-17,2056,2056,0,[]
62,LINKUSDT,2020-01-01,2025-08-17,2056,2056,0,[]
66,CRVUSDT,2020-08-15,2025-08-17,1829,1829,0,[]
69,PYTHUSDT,2024-02-02,2025-08-17,563,563,0,[]
71,ARBUSDT,2023-03-23,2025-08-17,879,879,0,[]


In [8]:
## check the number of symbols for bn future 1min bar for each date

In [12]:
path = '/data/crypto/bar1m/futures/'
files = os.listdir(path)
print(len(files))


471


In [13]:
rows = []

for symbol in files:
    df = pd.read_parquet(f"{path}/{symbol}")
    df["end_tm"] = pd.to_datetime(df["end_tm"])
    unique_dates = df["end_tm"].dt.normalize().unique()
    unique_dates = pd.to_datetime(unique_dates)

    start_dt = unique_dates.min()
    end_dt = unique_dates.max()

    full_range = pd.date_range(start=start_dt, end=end_dt, freq="D")

    missing = sorted(set(full_range) - set(unique_dates))

    rows.append({
        "coin": symbol[:-8],   # strip .parquet
        "start_dt": start_dt,
        "end_dt": end_dt,
        "unique_days": len(unique_dates),
        "expected_days": len(full_range),
        "missing_days": len(missing),
        "missing_dates": [d.strftime("%Y-%m-%d") for d in missing]  # readable
    })b

df_summary = pd.DataFrame(rows)



In [14]:
df_summary[df_summary.missing_days!=0]

,coin,start_dt,end_dt,unique_days,expected_days,missing_days,missing_dates
7,TRXUSDT,2020-01-15,2025-08-17,2037,2042,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
15,VETUSDT,2020-02-14,2025-07-17,1976,1981,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
22,IOTAUSDT,2020-02-12,2025-07-17,1978,1983,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
42,KNCUSDT,2020-06-22,2025-07-17,1847,1852,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
83,XRPUSDT,2020-01-06,2025-08-17,2046,2051,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
96,GMTUSDT,2022-03-15,2025-07-17,1219,1221,2,"[2022-04-01, 2022-04-02]"
98,FLMUSDT,2020-09-29,2025-07-17,1748,1753,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
116,STORJUSDT,2020-09-16,2025-07-17,1761,1766,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
132,ZECUSDT,2020-02-05,2025-07-17,1985,1990,5,"[2022-02-26, 2022-02-27, 2022-02-28, 2022-04-0..."
149,TLMUSDT,2021-07-16,2025-07-17,1434,1463,29,"[2023-03-01, 2023-03-02, 2023-03-03, 2023-03-0..."


In [15]:
# 
import cfg
from bn_downloader import *
from datetime import datetime
start = pd.to_datetime('2020-01-01')
end = datetime.now()
data_dir = '/data/crypto/'
for idx, row in df_summary[df_summary.missing_days!=0].iterrows():
    #break
    dates = row['missing_dates']
    download_daily_klines('um', [row['coin']], 1, ['1m'], dates, start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'), cfg.bar1m_folder, True)
    

Found 1 symbols
[1/1] - start download daily TRXUSDT klines 

File Download: /data/crypto/bar1m/data/futures/um/daily/klines/TRXUSDT/1m/TRXUSDT-1m-2022-02-26.zip
[##################################################]
File Download: /data/crypto/bar1m/data/futures/um/daily/klines/TRXUSDT/1m/TRXUSDT-1m-2022-02-26.zip.CHECKSUM
[##################################################]
File Download: /data/crypto/bar1m/data/futures/um/daily/klines/TRXUSDT/1m/TRXUSDT-1m-2022-02-27.zip
[##################################################]
File Download: /data/crypto/bar1m/data/futures/um/daily/klines/TRXUSDT/1m/TRXUSDT-1m-2022-02-27.zip.CHECKSUM
[##################################################]
File Download: /data/crypto/bar1m/data/futures/um/daily/klines/TRXUSDT/1m/TRXUSDT-1m-2022-02-28.zip
[##################################################]
File Download: /data/crypto/bar1m/data/futures/um/daily/klines/TRXUSDT/1m/TRXUSDT-1m-2022-02-28.zip.CHECKSUM
[#############################################

In [29]:

KLINES_COLS = ["start_tm", "open","high","low","close","vol","end_tm", "quote","trades","buy_vol","buy_quote","ignore"]

for idx, row in df_summary[df_summary.missing_days!=0].iterrows():
    ins = row['coin']
    fpath = os.path.join('/data/crypto/bar1m', 'futures', f"{ins}.parquet")
    prev_data = pd.read_parquet(fpath)
    dates = row['missing_dates']
    fs = [f'/data/crypto/bar1m/data/futures/um/daily/klines/{ins}/1m/{ins}-1m-{date}.zip' for date in dates]
    res = pd.DataFrame()
    for ifile in fs:
        if not os.path.exists(ifile):
            print(f"[missing]: {ifile}")
            continue
        tmp = pd.read_csv(ifile, names=KLINES_COLS)     
        # Skip empty files
        if tmp.empty:
            continue  
        # Handle header row
        if tmp.loc[0, 'start_tm'] == 'open_time':
            tmp = tmp.drop(0)
        # Skip if no data after header removal
        if tmp.empty:
            continue          
        # Convert timestamps
        nlen = len(str(tmp.loc[tmp.index[0], 'start_tm']))
        if nlen > 16:
            tmp['start_tm'] = pd.to_datetime(tmp['start_tm'], unit="ns")
            tmp['end_tm'] = pd.to_datetime(tmp['end_tm'], unit="ns")
        elif nlen > 13:
            tmp['start_tm'] = pd.to_datetime(tmp['start_tm'], unit="us")
            tmp['end_tm'] = pd.to_datetime(tmp['end_tm'], unit="us")
        else:
            tmp['start_tm'] = pd.to_datetime(tmp['start_tm'], unit="ms")
            tmp['end_tm'] = pd.to_datetime(tmp['end_tm'], unit="ms")
        res = pd.concat([res, tmp], ignore_index=True)
    res = res.astype({
            "open": float, "high": float, "low": float, "close": float,
            "vol": float, "quote": float, "trades": int, 
            "buy_vol": float, "buy_quote": float, 'ignore': int
        })   
    res = pd.concat([prev_data, res], ignore_index=True)
    res = res.sort_values('end_tm').drop_duplicates(subset=['end_tm'])
    res.to_parquet(fpath)
    print(f"[complete]: {ins}")


[complete]: TRXUSDT
[complete]: VETUSDT
[complete]: IOTAUSDT
[complete]: KNCUSDT
[complete]: XRPUSDT
[complete]: GMTUSDT
[complete]: FLMUSDT
[complete]: STORJUSDT
[complete]: ZECUSDT


/tmp/ipykernel_3078076/1396960364.py:33: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  tmp['start_tm'] = pd.to_datetime(tmp['start_tm'], unit="ms")
/tmp/ipykernel_3078076/1396960364.py:34: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  tmp['end_tm'] = pd.to_datetime(tmp['end_tm'], unit="ms")
/tmp/ipykernel_3078076/1396960364.py:33: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime stri

[complete]: TLMUSDT
[complete]: SUSHIUSDT
[complete]: MASKUSDT
[complete]: SANDUSDT
[complete]: GRTUSDT
[complete]: MKRUSDT
[complete]: ONEUSDT
[complete]: FLOWUSDT
[complete]: HBARUSDT
[complete]: PEOPLEUSDT
[complete]: TRBUSDT
[complete]: GTCUSDT
[complete]: KSMUSDT
[complete]: RVNUSDT
[complete]: SKLUSDT
[complete]: ZENUSDT
[complete]: RLCUSDT
[complete]: YFIUSDT
[complete]: LTCUSDT
[complete]: ZILUSDT
[complete]: RSRUSDT


/tmp/ipykernel_3078076/1396960364.py:33: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  tmp['start_tm'] = pd.to_datetime(tmp['start_tm'], unit="ms")
/tmp/ipykernel_3078076/1396960364.py:34: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  tmp['end_tm'] = pd.to_datetime(tmp['end_tm'], unit="ms")
/tmp/ipykernel_3078076/1396960364.py:33: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime stri

[missing]: /data/crypto/bar1m/data/futures/um/daily/klines/ICPUSDT/1m/ICPUSDT-1m-2022-09-22.zip
[missing]: /data/crypto/bar1m/data/futures/um/daily/klines/ICPUSDT/1m/ICPUSDT-1m-2022-09-23.zip
[missing]: /data/crypto/bar1m/data/futures/um/daily/klines/ICPUSDT/1m/ICPUSDT-1m-2022-09-24.zip
[missing]: /data/crypto/bar1m/data/futures/um/daily/klines/ICPUSDT/1m/ICPUSDT-1m-2022-09-25.zip
[missing]: /data/crypto/bar1m/data/futures/um/daily/klines/ICPUSDT/1m/ICPUSDT-1m-2022-09-26.zip
[complete]: ICPUSDT
[complete]: HOTUSDT
[complete]: SOLUSDT
[complete]: FILUSDT
[complete]: MANAUSDT
[complete]: IMXUSDT
[complete]: NEARUSDT
[complete]: OGNUSDT
[complete]: LRCUSDT
[complete]: ROSEUSDT


In [12]:
df_summary[df_summary.coin.isin(COINS)].sort_values('missing_days')

,coin,start_dt,end_dt,unique_days,expected_days,missing_days,missing_dates
21,1000PEPEUSDT,2023-05-05,2025-08-17,836,836,0,[]
34,RENDERUSDT,2024-07-26,2025-08-17,388,388,0,[]
35,JUPUSDT,2024-01-31,2025-08-17,565,565,0,[]
70,INJUSDT,2022-08-17,2025-08-17,1097,1097,0,[]
84,LINKUSDT,2020-01-17,2025-08-17,2040,2040,0,[]
77,AXSUSDT,2020-11-20,2025-08-17,1732,1732,0,[]
92,PYTHUSDT,2023-11-22,2025-08-17,635,635,0,[]
88,CRVUSDT,2020-09-01,2025-08-17,1812,1812,0,[]
108,DOTUSDT,2020-08-22,2025-08-17,1822,1822,0,[]
111,APEUSDT,2022-03-17,2025-08-17,1250,1250,0,[]


In [13]:
## check the number of symbols for bn spot trades for each date

In [39]:
BASE_PATH = '/data/crypto/tardis/'
data_type = 'trades_spot_bn'
all_files = os.listdir(os.path.join(BASE_PATH, data_type))



In [40]:
all_files[0]

'binance_trades_2022-03-16_DOGEUSDT.csv.gz'

In [41]:
pattern = re.compile(r"(\d{4}-\d{2}-\d{2})_(.+)\.csv\.gz")

coin_dates = {}
for f in all_files:
    match = pattern.search(f)
    if match:
        dt, coin = match.groups()
        coin_dates.setdefault(coin, set()).add(pd.to_datetime(dt))

rows = []
for coin, available in coin_dates.items():
    available_sorted = sorted(available)
    start = available_sorted[0]
    end = available_sorted[-1]
    dt_range = pd.date_range(start=start, end=end, freq="D")

    total_days_available = len(available)
    total_days_expected = len(dt_range)
    missing = sorted(set(dt_range) - available)
    missing_days = len(missing)

    rows.append([
        coin,
        start, 
        end,
        total_days_available,
        total_days_expected,
        missing_days,
        missing
    ])

df_summary = pd.DataFrame(
    rows, 
    columns=["coin", "start_date", "end_date", "total_days", "expected_days", "missing_days", "missing_dates"]
)


In [42]:
df_summary[df_summary.coin.isin(COINS)].sort_values('missing_days')

,coin,start_date,end_date,total_days,expected_days,missing_days,missing_dates
2,PENDLEUSDT,2023-07-03,2025-08-17,777,777,0,[]
15,TRXUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
16,BNBUSDT,2022-01-01,2025-08-17,1325,1325,0,[]
25,AXSUSDT,2022-01-01,2025-08-17,1325,1325,0,[]
33,SOLUSDT,2022-01-01,2025-08-17,1325,1325,0,[]
30,STXUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
41,DYDXUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
43,LINKUSDT,2022-01-01,2025-08-17,1325,1325,0,[]
93,ATOMUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
65,LDOUSDT,2022-05-09,2025-08-18,1198,1198,0,[]


In [2]:

from tardis_wraper import download_data_from_tardis
futuniverse = pd.read_parquet(cfg.universe_folder + 'bn_future_universe.parquet')
spotuniverse = pd.read_parquet(cfg.universe_folder + 'bn_spot_universe.parquet')

/home/moneyking/projects/mlframework/data_downloader/tardis_wraper/data_loader.py:202: SyntaxWarning: invalid escape sequence '\.'
  fpatt = re.compile(r'.*binance_trades_(\d{4}-\d{2}-\d{2})_'+ symbol +'\.csv\.gz$')
/home/moneyking/projects/mlframework/data_downloader/tardis_wraper/data_loader.py:231: SyntaxWarning: invalid escape sequence '\.'
  fpatt = re.compile(r'.*binance-futures_trades_(\d{4}-\d{2}-\d{2})_'+ symbol +'\.csv\.gz$')
/home/moneyking/projects/mlframework/data_downloader/tardis_wraper/data_loader.py:260: SyntaxWarning: invalid escape sequence '\.'
  fpatt = re.compile(r'.*binance-futures_derivative_ticker_(\d{4}-\d{2}-\d{2})_'+ symbol +'\.csv\.gz$')


In [3]:
for idx, row in df_summary[df_summary.coin.isin(COINS)].iterrows():
    #break
    dates = row['missing_dates']
    if len(dates) > 0:
        sdate = min(dates).strftime("%Y-%m-%d")
        edate = max(dates).strftime("%Y-%m-%d")
        download_data_from_tardis(    
                sdate, edate,
                row['coin'],
                'trades',
                'binance',
                details = spotuniverse
            )

NameError: name 'df_summary' is not defined

In [1]:
download_data_from_tardis(    
                '2025-07-28', '2025-08-05',
                'DOGEUSDT',
                'trades',
                'binance',
                details = spotuniverse
            )

NameError: name 'download_data_from_tardis' is not defined

In [18]:
## check the number of symbols for bn fut trades for each date

In [6]:
BASE_PATH = '/data/crypto/tardis/'
data_type = 'trades_bn'
all_files = os.listdir(os.path.join(BASE_PATH, data_type))


In [20]:
all_files[0]

'binance-futures_trades_2023-08-18_STXUSDT.csv.gz'

In [7]:
pattern = re.compile(r"(\d{4}-\d{2}-\d{2})_(.+)\.csv\.gz")

coin_dates = {}
for f in all_files:
    match = pattern.search(f)
    if match:
        dt, coin = match.groups()
        coin_dates.setdefault(coin, set()).add(pd.to_datetime(dt))

rows = []
for coin, available in coin_dates.items():
    available_sorted = sorted(available)
    start = available_sorted[0]
    end = available_sorted[-1]
    dt_range = pd.date_range(start=start, end=end, freq="D")

    total_days_available = len(available)
    total_days_expected = len(dt_range)
    missing = sorted(set(dt_range) - available)
    missing_days = len(missing)

    rows.append([
        coin,
        start, 
        end,
        total_days_available,
        total_days_expected,
        missing_days,
        missing
    ])

df_summary = pd.DataFrame(
    rows, 
    columns=["coin", "start_date", "end_date", "total_days", "expected_days", "missing_days", "missing_dates"]
)


/home/moneyking/miniconda3/lib/python3.12/site-packages/dateutil/parser/_parser.py:201: RuntimeWarning: coroutine 'download_async' was never awaited
  return list(cls(s))


In [10]:
df_summary[df_summary.coin.isin(COINS)].sort_values('missing_days')

,coin,start_date,end_date,total_days,expected_days,missing_days,missing_dates
0,STXUSDT,2023-02-21,2025-08-20,912,912,0,[]
5,CRVUSDT,2022-01-01,2025-08-20,1328,1328,0,[]
6,DYDXUSDT,2022-01-01,2025-08-20,1328,1328,0,[]
13,FILUSDT,2022-01-01,2025-08-20,1328,1328,0,[]
20,BNBUSDT,2022-01-01,2025-08-20,1328,1328,0,[]
23,GALAUSDT,2022-01-01,2025-08-20,1328,1328,0,[]
31,INJUSDT,2022-08-17,2025-08-20,1100,1100,0,[]
41,NEARUSDT,2022-01-01,2025-08-20,1328,1328,0,[]
53,OPUSDT,2022-06-01,2025-08-20,1177,1177,0,[]
54,ATOMUSDT,2022-01-01,2025-08-20,1328,1328,0,[]


In [ ]:
for idx, row in df_summary[df_summary.coin.isin(COINS)].iterrows():
    #break
    dates = row['missing_dates']
    if len(dates) > 0:
        sdate = min(dates).strftime("%Y-%m-%d")
        edate = max(dates).strftime("%Y-%m-%d")
        download_data_from_tardis(    
                sdate, edate,
                row['coin'],
                'trades',
                'binance-futures',
                details = futuniverse
            )


In [22]:
df_summary

,coin,start_date,end_date,total_days,expected_days,missing_days,missing_dates
0,STXUSDT,2023-02-21,2025-08-19,911,911,0,[]
1,MOCAUSDT,2024-12-16,2025-08-05,233,233,0,[]
2,SUPERUSDT,2024-01-01,2025-08-05,583,583,0,[]
3,VETUSDT,2022-01-01,2025-08-19,1327,1327,0,[]
4,CTSIUSDT,2022-01-01,2025-08-05,1313,1313,0,[]
...,...,...,...,...,...,...,...
481,IDOLUSDT,2025-07-04,2025-08-05,33,33,0,[]
482,RESOLVUSDT,2025-06-10,2025-08-05,57,57,0,[]
483,ZKJUSDT,2025-05-13,2025-08-05,85,85,0,[]
484,HNTUSDT,2023-01-01,2023-01-01,1,1,0,[]


In [24]:
## check the number of symbols for bn derivative ticker for each date

In [11]:
BASE_PATH = '/data/crypto/tardis/'
data_type = 'derivative_ticker_bn'
all_files = os.listdir(os.path.join(BASE_PATH, data_type))


In [26]:
all_files[0]

'binance-futures_derivative_ticker_2024-05-14_LRCUSDT.csv.gz'

In [12]:
pattern = re.compile(r"(\d{4}-\d{2}-\d{2})_(.+)\.csv\.gz")

coin_dates = {}
for f in all_files:
    match = pattern.search(f)
    if match:
        dt, coin = match.groups()
        coin_dates.setdefault(coin, set()).add(pd.to_datetime(dt))

rows = []
for coin, available in coin_dates.items():
    available_sorted = sorted(available)
    start = available_sorted[0]
    end = available_sorted[-1]
    dt_range = pd.date_range(start=start, end=end, freq="D")

    total_days_available = len(available)
    total_days_expected = len(dt_range)
    missing = sorted(set(dt_range) - available)
    missing_days = len(missing)

    rows.append([
        coin,
        start, 
        end,
        total_days_available,
        total_days_expected,
        missing_days,
        missing
    ])

df_summary = pd.DataFrame(
    rows, 
    columns=["coin", "start_date", "end_date", "total_days", "expected_days", "missing_days", "missing_dates"]
)


/home/moneyking/miniconda3/lib/python3.12/site-packages/dateutil/parser/_parser.py:201: RuntimeWarning: coroutine 'download_async' was never awaited
  return list(cls(s))


In [13]:
df_summary[df_summary.coin.isin(COINS)].sort_values('missing_days')

,coin,start_date,end_date,total_days,expected_days,missing_days,missing_dates
1,DOTUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
8,SEIUSDT,2023-08-17,2025-08-18,733,733,0,[]
14,ETCUSDT,2022-01-01,2025-08-17,1325,1325,0,[]
18,APEUSDT,2022-03-17,2025-08-18,1251,1251,0,[]
35,BNBUSDT,2022-01-01,2025-08-17,1325,1325,0,[]
24,ARBUSDT,2023-03-23,2025-08-17,879,879,0,[]
37,AVAXUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
52,DYDXUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
65,TRXUSDT,2022-01-01,2025-08-18,1326,1326,0,[]
54,GALAUSDT,2022-01-01,2025-08-17,1325,1325,0,[]


In [ ]:
for idx, row in df_summary[df_summary.coin.isin(COINS)].iterrows():
    #break
    dates = row['missing_dates']
    if len(dates) > 0:
        sdate = min(dates).strftime("%Y-%m-%d")
        edate = max(dates).strftime("%Y-%m-%d")
        download_data_from_tardis(    
                sdate, edate,
                row['coin'],
                'derivative_ticker',
                'binance-futures',
                details = futuniverse
            )


In [29]:
## check the number of symbols for bn liquidation for each date

In [30]:
BASE_PATH = '/data/crypto/tardis/'
data_type = 'liquidations'
all_files = os.listdir(os.path.join(BASE_PATH, data_type))


In [31]:
all_files[0]

'binance-futures_liquidations_2025-02-21_MEMEUSDT.csv.gz'

In [32]:
pattern = re.compile(r"(\d{4}-\d{2}-\d{2})_(.+)\.csv\.gz")

coin_dates = {}
for f in all_files:
    match = pattern.search(f)
    if match:
        dt, coin = match.groups()
        coin_dates.setdefault(coin, set()).add(pd.to_datetime(dt))

rows = []
for coin, available in coin_dates.items():
    available_sorted = sorted(available)
    start = available_sorted[0]
    end = available_sorted[-1]
    dt_range = pd.date_range(start=start, end=end, freq="D")

    total_days_available = len(available)
    total_days_expected = len(dt_range)
    missing = sorted(set(dt_range) - available)
    missing_days = len(missing)

    rows.append([
        coin,
        start, 
        end,
        total_days_available,
        total_days_expected,
        missing_days,
        missing
    ])

df_summary = pd.DataFrame(
    rows, 
    columns=["coin", "start_date", "end_date", "total_days", "expected_days", "missing_days", "missing_dates"]
)


In [33]:
df_summary[df_summary.coin.isin(COINS)].sort_values('missing_days')

,coin,start_date,end_date,total_days,expected_days,missing_days,missing_dates
4,INJUSDT,2024-01-01,2025-08-18,596,596,0,[]
6,SEIUSDT,2024-01-01,2025-08-18,596,596,0,[]
21,1000SHIBUSDT,2024-01-01,2025-08-17,595,595,0,[]
30,1000BONKUSDT,2024-01-01,2025-08-17,595,595,0,[]
53,APEUSDT,2024-01-01,2025-08-18,596,596,0,[]
37,FETUSDT,2024-01-01,2025-08-17,595,595,0,[]
57,1000FLOKIUSDT,2024-01-01,2025-08-17,595,595,0,[]
64,BCHUSDT,2024-01-01,2025-08-18,596,596,0,[]
83,AXSUSDT,2024-01-01,2025-08-17,595,595,0,[]
72,JTOUSDT,2024-01-01,2025-08-18,596,596,0,[]


In [ ]:
download_data_from_tardis(    
                '2024-01-01', '2025-08-17',
                'ARUSDT',
                'liquidations',
                'binance-futures',
                details = futuniverse
            )

In [46]:
BASE_PATH = '/data/crypto/tardis/'
data_type = 'liquidations'
all_files = os.listdir(os.path.join(BASE_PATH, data_type))

pattern = re.compile(r"(\d{4}-\d{2}-\d{2})_ETHUSDT\.csv\.gz")

coin_dates = {}
for f in all_files:
    match = pattern.search(f)
    if match:
        dt = match.groups()[0]
        coin_dates.setdefault('ETHUSDT', set()).add(pd.to_datetime(dt))


rows = []
for coin, available in coin_dates.items():
    available_sorted = sorted(available)
    start = available_sorted[0]
    end = available_sorted[-1]
    dt_range = pd.date_range(start=start, end=end, freq="D")

    total_days_available = len(available)
    total_days_expected = len(dt_range)
    missing = sorted(set(dt_range) - available)
    missing_days = len(missing)

    rows.append([
        coin,
        start, 
        end,
        total_days_available,
        total_days_expected,
        missing_days,
        missing
    ])

pd.DataFrame(
    rows, 
    columns=["coin", "start_date", "end_date", "total_days", "expected_days", "missing_days", "missing_dates"]
)


,coin,start_date,end_date,total_days,expected_days,missing_days,missing_dates
0,ETHUSDT,2022-01-01,2025-08-17,1306,1325,19,"[2023-12-28 00:00:00, 2023-12-29 00:00:00, 202..."


In [49]:
list(pd.DataFrame(
    rows, 
    columns=["coin", "start_date", "end_date", "total_days", "expected_days", "missing_days", "missing_dates"]
)['missing_dates'])

[[Timestamp('2023-12-28 00:00:00'),
  Timestamp('2023-12-29 00:00:00'),
  Timestamp('2023-12-30 00:00:00'),
  Timestamp('2023-12-31 00:00:00'),
  Timestamp('2025-07-22 00:00:00'),
  Timestamp('2025-07-23 00:00:00'),
  Timestamp('2025-07-24 00:00:00'),
  Timestamp('2025-07-25 00:00:00'),
  Timestamp('2025-07-26 00:00:00'),
  Timestamp('2025-07-27 00:00:00'),
  Timestamp('2025-07-28 00:00:00'),
  Timestamp('2025-07-29 00:00:00'),
  Timestamp('2025-07-30 00:00:00'),
  Timestamp('2025-07-31 00:00:00'),
  Timestamp('2025-08-01 00:00:00'),
  Timestamp('2025-08-02 00:00:00'),
  Timestamp('2025-08-03 00:00:00'),
  Timestamp('2025-08-04 00:00:00'),
  Timestamp('2025-08-05 00:00:00')]]

In [50]:
start

Timestamp('2022-01-01 00:00:00')